# Datas

> this notebook retrives at first all the edita that has been classified as weaponisation by the LLM and by Hamest

In [13]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
# Path to the folder containing CSVs or .xlsx
csv_folder = Path("../Hedi/LLM Results")
hamest_file = Path("../Hedi/hamest.xlsx")

# Create a list of DataFrames, one for each CSV
dfs = [pd.read_csv(f) for f in csv_folder.glob("*.csv")]
hamest = pd.read_excel(hamest_file)

dfs_named = {f.stem: pd.read_csv(f) for f in csv_folder.glob("*.csv")}

# Example: show how many DataFrames were loaded
print(f"Loaded {len(dfs)} CSV files.")
print("Available keys:", list(dfs_named.keys()))

Loaded 120 CSV files.
Available keys: ['COVID-19_pandemic_in_Ukraine_analysis', 'Immigration_to_Ukraine_analysis', 'Andrei_Sheptytsky_analysis', 'Football_in_Ukraine_analysis', 'Abortion_in_Ukraine_analysis', '2004_Ukrainian_presidential_election_analysis', 'History_of_Ukraine_analysis', 'Christianity_in_Russia_analysis', 'Cossack’s_songs_of_Dnipropetrovsk_Region_analysis', 'Flag_of_Ukraine_analysis', 'Belarusians_in_Ukraine_analysis', 'Cinema_of_Ukraine_analysis', 'Brotherhood_of_Independent_Baptist_Churches_and_Ministries_of_Ukraine_analysis', 'Hillsong_Ukraine_analysis', 'Communist_Party_of_the_Soviet_Union_analysis', 'Electricity_in_Ukraine_analysis', 'Censuses_in_Ukraine_analysis', 'Declaration_of_Independence_of_Ukraine_analysis', 'History_of_Christianity_in_Ukraine_analysis', 'Bessarabia_analysis', 'Human_trafficking_in_Ukraine_analysis', 'Crimea_analysis', 'Epiphanius_I_of_Ukraine_analysis', 'Hillsong_Church_Kiev_analysis', 'Government_of_the_Ukrainian_People_s_Republic_in_exil

In [5]:
dfs_named["2004_Ukrainian_presidential_election_analysis"].sample()
dfs_named["Abortion_in_Ukraine_analysis"].shape

(47, 7)

In [4]:
lines = 0
for df in dfs:
    lines += df.shape[0]
print(lines)

117516


In [2]:
# dfs_weaponised is a list of the dataframes with only edits that were classified as weaponised by the LLM
dfs_weaponised = [df[df['weaponised'] == 'Weaponised'] for df in dfs]

In [6]:
lines = 0
for df in dfs_weaponised:
    lines += df.shape[0]
print(lines)
# We have now 18707 rows that are classfied as weaponsied --> 18707 potential users.

18707


In [7]:
dfs_weaponised[0].sample()

,initial_version,changed_version,comment,user,date,llm_output,weaponised
1658,{{main|2020 coronavirus outbreak in Europe}}\n...,"--- \n+++ \n@@ -155,7 +155,7 @@\n \n === Russi...","Rescuing orphaned refs ("":1"" from rev 1090597485)",AnomieBOT,2022-05-30T10:45:06Z,The change made in this revision is the additi...,Weaponised


In [32]:
hamest_new = hamest.iloc[:, :-2]
hamest_new = hamest_new.rename(columns={"Annotation_1": "weaponised"})
hamest_new_weaponised = hamest_new[hamest_new['weaponised'] == 'Weaponised']
hamest_new_weaponised.shape # (122, 5)
hamest_new_weaponised.head()

,Source,JSON_Record,Analysis,Judgment,weaponised
0,Kyiv_Pechersk_Lavra.jsonl,"{""version"": ""diff"", ""Timestamp"": ""2019-12-10T1...",Record 594 (Version: diff):\n**Judgment: Weapo...,Weaponised,Weaponised
3,Kyiv_Pechersk_Lavra.jsonl,"{""version"": ""diff"", ""Timestamp"": ""2020-09-18T1...",Record 646 (Version: diff):\n**Judgment: Weapo...,Weaponised,Weaponised
4,History_of_Christianity_in_Ukraine.jsonl,"{""version"": ""diff"", ""Timestamp"": ""2018-04-17T2...",Record 956 (Version: diff):\n**Judgment: Weapo...,Weaponised,Weaponised
5,Talk_History_of_Christianity_in_Ukraine.jsonl,"{""version"": ""diff"", ""Timestamp"": ""2007-03-05T0...",Record 291 (Version: diff):\n**Judgment: Weapo...,Weaponised,Weaponised
9,History_of_Christianity_in_Ukraine.jsonl,"{""version"": ""diff"", ""Timestamp"": ""2006-03-07T0...",Record 141 (Version: diff):\n**Judgment: Weapo...,Weaponised,Weaponised


In [33]:
import json

users = [
    json.loads(record).get("User")
    for record in hamest_new_weaponised["JSON_Record"]
    if pd.notna(record)
]

print(users[:20])

['Nepsis2', 'Materialscientist', 'Aleksandr Grigoryev', 'Irpen', 'Ukrained', 'Jumbo T', '119.94.109.231', 'Truthseeker 85.5', 'Irpen', 'Blobmob', 'Sirfurboy', 'Ушкуйник', 'Kuban kazak', 'Kuban kazak', 'USchick', 'UncleDouggie', '217.66.159.71', 'Kuban kazak', 'ClueBot', '79.66.27.98']


## Details about Wikipedia's users 

- User can be an IP adress --> IP users, guest users, or unregistered users are users who edit Wikipedia without registering for an account.
- User can be a Bot --> automatic editing
- User can be a registred user

### Bots

Let's focus on Bot at first because it feels weird that bots can edit Wikipedia's article. Apparently, bots are registered and approved. They must :
* Have a separate, clearly labeled account (usually ending with “Bot”, like ClueBot NG).
* Get approval from the community through the “Bot Approvals Group” process.
* Be limited in scope to well-defined, repetitive tasks (like fixing typos, reverting vandalism, adding categories, or updating statistics).
So, bots don’t randomly modify articles — they work under strict rules.

> Are bots powered by Mediwiki or can anyone create a bot and make it work on wikipedia's article via API ?